In [1]:
import pandas as pd 

import pandas as pd

from sklearn.model_selection import train_test_split

from kmodes.kmodes import KModes
from sklearn.model_selection import train_test_split
from pycaret.classification import *

%run class/patient_details.py
%run util/utils.py

Cardiovascular Disease Predictor

In [2]:
#Cardiovascular
cardio_df = pd.read_csv("data/cardio_clean.csv")

df_male = cardio_df.query("gender == 1")
df_female = cardio_df.query("gender == 0")

# female data
# Building the model with using K-Mode with "Huang" initialization
km_huang = KModes(n_clusters=2, init = "Huang", n_init = 5, verbose=0)

male_kmode = km_huang.fit(df_male)
female_kmode = km_huang.fit(df_female)

clusters_female = km_huang.fit_predict(df_female)
clusters_male = km_huang.fit_predict(df_male)

# female DataFrame with Clusters
df_female.insert(0,"cluster", clusters_female, True)

# male DataFrame with Clusters
df_male.insert(0, "cluster", clusters_male, True)

# replacing cluster column values to merge dataframes after
df_male["cluster"].replace({0:2, 1:3}, inplace=True)

# merging female and male data
df_clusters = pd.concat([df_female, df_male], ignore_index=True, sort=False)

clf1 = setup(data = df_clusters, 
             target = 'cardio',
             preprocess = False,
             silent = True)

models()

top5_cvd = compare_models(sort='AUC',
                    n_select = 5,
                 )
stack_model_cvd = stack_models(estimator_list = top5_cvd, meta_model = top5_cvd[0], optimize = 'AUC')

final_model_cvd = finalize_model(stack_model_cvd)




,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8751,0.9522,0.8528,0.8865,0.8693,0.7497,0.7502
1,0.8644,0.9485,0.8548,0.8653,0.8600,0.7285,0.7285
2,0.8568,0.9446,0.8328,0.8680,0.8500,0.7131,0.7136
3,0.8703,0.9520,0.8493,0.8802,0.8645,0.7402,0.7407
4,0.8646,0.9462,0.8294,0.8855,0.8565,0.7286,0.7300
5,0.8705,0.9516,0.8430,0.8858,0.8639,0.7406,0.7414
6,0.8637,0.9429,0.8450,0.8713,0.8579,0.7269,0.7272
7,0.8603,0.9444,0.8430,0.8667,0.8547,0.7203,0.7205
8,0.8625,0.9468,0.8342,0.8774,0.8553,0.7244,0.7253


Diabetes Predictor

In [3]:
#Diabetes
diabetes_df = pd.read_csv('data/diabetes_clean.csv')
diabetes_df.info()

X = diabetes_df.iloc[:,:-1]
y = diabetes_df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

clf2 = setup(data = diabetes_df, 
             target = 'Outcome',
             preprocess = False,
             silent = True)
             
models()

top5_diabetes = compare_models(sort='AUC', n_select = 5)

blend_soft_diabetes = blend_models(estimator_list = top5_diabetes, optimize = 'AUC',method = 'soft')

cali_model_diabetes = calibrate_model(blend_soft_diabetes)

final_model_diabetes = finalize_model(cali_model_diabetes)



,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7407,0.8721,0.5000,0.7143,0.5882,0.4075,0.4213
1,0.6852,0.6824,0.4500,0.6000,0.5143,0.2884,0.2949
2,0.7778,0.9074,0.5000,0.8333,0.6250,0.4808,0.5125
3,0.8333,0.9132,0.7000,0.8235,0.7568,0.6313,0.6361
4,0.6481,0.7574,0.4000,0.5333,0.4571,0.2047,0.2093
5,0.8519,0.9176,0.8500,0.7727,0.8095,0.6888,0.6909
6,0.7407,0.8044,0.6500,0.6500,0.6500,0.4441,0.4441
7,0.7925,0.9025,0.6842,0.7222,0.7027,0.5435,0.5439
8,0.7547,0.8746,0.5263,0.7143,0.6061,0.4339,0.4445


Reusable methods

In [4]:
def predict_diabetes(input_dictionary):
    prediction_diabetes = final_model_diabetes.predict([[
        input_dictionary["pregnancies"],
        input_dictionary["glucose"],
        input_dictionary["diastolic_bp"],
        input_dictionary["skin_thickness"],
        input_dictionary["insulin"],
        input_dictionary["bmi"],
        input_dictionary["dpf"],
        input_dictionary["age"]
    ]])
    return prediction_diabetes[0]

def predict_cvd(input_dictionary):
    prediction_cvd = final_model_cvd.predict([[
        input_dictionary["cluster"],
        input_dictionary["gender"],
        input_dictionary["age_class"],
        input_dictionary["bmi_class"],
        input_dictionary["map_class"],
        input_dictionary["cholesterol_class"],
        input_dictionary["glucose_class"],
        input_dictionary["smoke"],
        input_dictionary["active"]
    ]])
    return prediction_cvd[0]
 
def assign_cluster_male(patient):
    cluster = male_kmode.predict(patient)
    if cluster[0] == 0: return 2
    if cluster[0] == 1: return 3 

def assign_cluster_female(patient):
    predict = female_kmode.predict(patient)
    return predict[0]

def assign_cluster(input_dictionary):
    if input_dictionary["gender"] == 0: #Female
        return assign_cluster_female([[
            input_dictionary["gender"],
            input_dictionary["age_class"],
            input_dictionary["bmi_class"],
            input_dictionary["map_class"],
            input_dictionary["cholesterol_class"],
            input_dictionary["glucose_class"],
            input_dictionary["smoke"],
            input_dictionary["active"],
            input_dictionary["cvd_prediction"]
        ]])
    else:   
        return assign_cluster_male([[
            input_dictionary["gender"],
            input_dictionary["age_class"],
            input_dictionary["bmi_class"],
            input_dictionary["map_class"],
            input_dictionary["cholesterol_class"],
            input_dictionary["glucose_class"],
            input_dictionary["smoke"],
            input_dictionary["active"],
            input_dictionary["cvd_prediction"]
        ]])

Web Application Implementation

In [50]:
%run class/patient_details.py
%run util/utils.py

#Dummy input assuming
#Assuming these are inputs from the web app
input_dictionary = {
    "age": 30, 
    "gender" : 0, 
    "glucose" : 190, 
    "cholesterol" : 200,
    "systolic_bp" : 150, 
    "diastolic_bp" : 100,
    "pregnancies" : 0,
    "skin_thickness" : 20,
    "insulin" : 300,
    "bmi" : 25,
    "dpf" : 1,
    "smoke" : 0,
    "active" : 1,
    "cvd_prediction" : 0, #Initial value is 0
    "diabetes_prediction" : 0 #Initial value is 0
}

input_dictionary["mean_arterial_pressure"] = compute_mean_arterial_pressure(input_dictionary["systolic_bp"], input_dictionary["diastolic_bp"])

#age class or age range
input_dictionary["age_class"] = get_age_class(input_dictionary["age"])

#assign classes to health vitals
input_dictionary["bmi_class"] = get_bmi_class(input_dictionary["bmi"])
input_dictionary["map_class"] = get_map_class(input_dictionary["mean_arterial_pressure"])
input_dictionary["cholesterol_class"] = get_cholesterol_class(input_dictionary["cholesterol"])
input_dictionary["glucose_class"] = get_glucose_class(input_dictionary["glucose"])
input_dictionary["cluster"] = assign_cluster(input_dictionary)
print("CVD PREDICT: ", predict_cvd(input_dictionary))
print("Diabetes PREDICT: ", predict_diabetes(input_dictionary))
input_dictionary["cvd_prediction"] = predict_cvd(input_dictionary)
input_dictionary["diabetes_prediction"] = predict_diabetes(input_dictionary)

patient = Patient(cluster = input_dictionary["cluster"], 
                  age = input_dictionary["age"],
                  gender = input_dictionary["gender"],
                  glucose = input_dictionary["glucose"],
                  cholesterol = input_dictionary["cholesterol"],
                  systolic_bp = input_dictionary["systolic_bp"],
                  diastolic_bp = input_dictionary["diastolic_bp"],
                  mean_arterial_pressure =  input_dictionary["mean_arterial_pressure"],
                  pregnancies = input_dictionary["pregnancies"],
                  skin_thickness = input_dictionary["skin_thickness"],
                  insulin = input_dictionary["insulin"],
                  bmi = input_dictionary["bmi"],
                  dpf = input_dictionary["dpf"], # Diabetes pedigree function
                  age_class = input_dictionary["age_class"],
                  bmi_class = input_dictionary["bmi_class"],
                  map_class = input_dictionary["map_class"],
                  cholesterol_class = input_dictionary["cholesterol_class"],
                  glucose_class = input_dictionary["glucose_class"],
                  smoke = input_dictionary["smoke"],
                  active = input_dictionary["active"],
                  cvd_prediction = input_dictionary["cvd_prediction"],
                  diabetes_prediction = input_dictionary["diabetes_prediction"]
                  )

patient_dictionary = patient.map_patient_details()
print(patient_dictionary)

CVD PREDICT:  1
Diabetes PREDICT:  1
{'cluster': 0, 'age': 30, 'gender': 0, 'cholesterol': 200, 'systolic_bp': 150, 'diastolic_bp': 100, 'mean_arterial_pressure': 116.66666666666667, 'skin_thickness': 20, 'insulin': 300, 'bmi': 25, 'dpf': 1, 'age_class': 0, 'bmi_class': 2, 'map_class': 3, 'cholesterol_class': 0, 'glucose_class': 2, 'smoke': 0, 'active': 1, 'cvd_prediction': 1, 'diabetes_prediction': 1}
